### 第22天：模型部署概述

今天的任務將帶你了解如何使用 FastAPI 建立一個 REST API 來部署你的深度學習模型，並學習如何使用 Docker 容器化你的應用程序以便於部署。

#### **FastAPI 概述**

**FastAPI** 是一個高效、現代的 Python Web 框架，適合用來構建 RESTful API，特別是在需要快速處理請求或與模型進行推理的情境中。

**為什麼選擇 FastAPI：**
- **高效**：FastAPI 使用的是 Python 的新特性，例如異步 I/O (asyncio)，使其具備優異的性能。
- **簡單明瞭**：它的語法和設計簡單易用，便於編寫和維護。
- **自動化文檔**：內建 Swagger UI 和 ReDoc，可輕鬆生成互動式 API 文檔，這對於開發和測試非常有用。

#### **部署流程概述**
1. **創建 FastAPI 應用**：創建一個 FastAPI 應用，並定義一個端點來接受圖片或數據，然後利用模型進行推理，最後返回結果。
2. **使用 Docker 容器化應用**：將 FastAPI 應用容器化，打包成一個可以跨平台運行的 Docker 映像。這樣可以確保在任何環境下運行都能一致且可靠。
3. **運行容器**：利用 Docker 運行 FastAPI 應用，並通過暴露的端口訪問 API。

#### **步驟 1：構建 FastAPI 應用**

```bash
pip install fastapi uvicorn
```

在專案目錄下創建一個 `main.py` 檔案：

```python
from fastapi import FastAPI, UploadFile, File
import uvicorn
from PIL import Image
import io
import torch

# 初始化 FastAPI 應用
app = FastAPI()

# 載入模型（例如 YOLO 模型）
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')  # 修改為你的模型路徑

# 定義一個端點來接受圖片並進行推理
@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = Image.open(io.BytesIO(await file.read()))
    results = model(image)
    return results.pandas().xyxy[0].to_dict(orient="records")  # 回傳物件檢測結果

# 運行 FastAPI 應用
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
```

這個程式碼會啟動一個 FastAPI 應用，並定義一個 `/predict/` 端點，讓你可以上傳圖片，並使用 YOLO 模型進行物件檢測，最後返回結果。

#### **步驟 2：使用 Docker 容器化應用**

首先，創建一個 `Dockerfile`，如下所示：

```dockerfile
# 使用官方 Python 基礎映像
FROM python:3.9-slim

# 設置工作目錄
WORKDIR /app

# 複製當前目錄到容器中
COPY . .

# 安裝必要的 Python 套件
RUN pip install --no-cache-dir fastapi uvicorn torch pillow

# 暴露 8000 端口
EXPOSE 8000

# 運行 FastAPI 應用
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

這個 `Dockerfile` 會創建一個基於 Python 3.9 的映像，並安裝必要的依賴庫，最後啟動 FastAPI 應用。

#### **步驟 3：構建 Docker 映像並運行**

1. 構建 Docker 映像：
   在專案目錄下運行以下命令來構建 Docker 映像：
   ```bash
   docker build -t fastapi-yolo-app .
   ```

2. 運行容器：
   使用以下命令運行容器：
   ```bash
   docker run -d --name fastapi-yolo -p 8000:8000 fastapi-yolo-app
   ```

這樣，你的 FastAPI 應用就會在 8000 端口上運行，可以通過瀏覽器或 Postman 來測試 API，發送圖片進行物件檢測。

#### **學習重點**
- **FastAPI** 是如何構建 API 的。
- **Docker** 是如何將你的應用容器化的。
- 如何在容器中運行深度學習模型並通過 API 提供推理服務。

這是模型部署的第一步，隨後你會學習到更多進階的部署技巧，包括雲端部署和模型監控等。